In [1]:
# from tile_bank_repository import TileBankRepository
from repository import TileBankRepository
from datetime import datetime
import numpy as np

## Initialize TileBank Repository

In [2]:
repo = TileBankRepository(save_dir="uploads")

In [3]:
for table in repo.list_tables():
    print(table)
    print(repo.get_table_schema(table))
    print("--------------------------------")

mask
         column_name                                        column_type null  \
0                 id                                            INTEGER   NO   
1               task  ENUM('ntp', 'field_delineation', 'perm_structu...   NO   
2               path                                            VARCHAR   NO   
3        path_border                                            VARCHAR  YES   
4      path_distance                                            VARCHAR  YES   
5            tile_id                                            INTEGER   NO   
6       date_created                                          TIMESTAMP   NO   
7        date_origin                                               DATE  YES   
8          mask_type                                            VARCHAR   NO   
9      timeseries_id                                            INTEGER  YES   
10            bounds                                           GEOMETRY   NO   
11  raster_transform               

In [4]:
repo.get_all("satellite", ["name", "type"])

#repo.get_all("satellite2") # should raise an error since the table does not exist

,name,type
0,Sentinel-2,optic
1,Sentinel-1,radar
2,Pleiades-50,optic
3,PleiadesNEO,optic
4,ortophoto25,optic


In [5]:
repo.find("satellite", name="Sentinel-2")

,id,name,resolution_cm,type,date_created
0,1,Sentinel-2,100,optic,2025-07-11 17:36:02.513


In [6]:
repo.get_by_id("satellite", 4)

#repo.get_by_id("satellite", 56) # should raise an error since the record does not exist

,id,name,resolution_cm,type,date_created
0,4,PleiadesNEO,30,optic,2025-07-11 17:36:02.524


## Adding new records

We can add new records to the db that are saved in files already like so:

In [7]:
repo.add_single_tile_from_path("0_4736.npy", "Sentinel-2", datetime.now())

# second attempt to add the same tile should raise an error
# repo.add_single_tile_from_path("0_47362.npy", "Sentinel-2", datetime.now())

# providing an invalid path should raise an error
#repo.add_single_tile_from_path("invalid_path.npy", "Sentinel-2", datetime.now())

# providing an invalid satellite name should raise an error
#repo.add_single_tile_from_path("0_4736.npy", "Invalid-Satellite", datetime.now())

# providing an invalid date should raise an error
#repo.add_tile("0_4736.npy", "Sentinel-2", "invalid_date")


ValueError: File 0_4736.npy does not exist

In [8]:
repo.get_by_id("tile", 1)

,id,path,satellite_id,date_created,date_origin
0,1,0_4736.npy,1,2025-07-11 17:41:57.399,2025-07-11


We can also give a numpy array and have the repository save it dynamically

In [9]:
dummy_array = np.random.rand(100, 100)

repo.add_single_tile_from_array(dummy_array, "Sentinel-2", datetime.now())

,id,path,satellite_id,date_created,date_origin
0,2,.\ea44132f-53f7-4215-8fa0-497cafb1d4fa.npy,1,2025-07-11 17:41:57.399,2025-07-11


In [ ]:
repo.add_timeseries_from_path(
    paths=["timestep_1.npy", "timestep_2.npy"],
    satellite_name="Sentinel-2",
    date_origins=[datetime.now(), datetime.now()]
)

,id,start_date,end_date,date_created
0,1,2025-07-11,2025-07-11,2025-07-11 17:41:57.399


In [10]:
repo.save_changes()

In [ ]:
from datetime import timedelta


dummy_high_res_data = np.random.rand(3, 100, 100)
dummy_timeseries_data = np.random.rand(2, 3, 100, 100)

repo.add_multimodal_from_array(dummy_high_res_data, dummy_timeseries_data, "Sentinel-2", datetime.now(), [datetime.now() + timedelta(days=1), datetime.now() + timedelta(days=2)])

2025-07-15
   id                                        path  satellite_id  \
0   1  .\c5fe1363-40b0-41a0-8724-6b65643c6e4c.npy             1   

             date_created date_origin  
0 2025-07-15 15:00:58.742  2025-07-15  
2025-07-16
   id                                        path  satellite_id  \
0   2  .\9387c7fe-55a8-4033-a2b2-50a487733c03.npy             1   

             date_created date_origin  
0 2025-07-15 15:00:58.742  2025-07-16  
2025-07-17
   id                                        path  satellite_id  \
0   3  .\f939ec44-9aad-4f41-a711-4a3f5ebe7dd6.npy             1   

             date_created date_origin  
0 2025-07-15 15:00:58.742  2025-07-17  


,id,timeseries_id,high_resolution_id,date_created
0,1,1,1,2025-07-15 15:00:58.742


In [ ]:
# Example 2: Adding a tile from a numpy array
# When working with arrays, you need to provide the spatial information
array = np.random.rand(4, 256, 256)  # Example array with 4 bands, 256x256 pixels

# Define the spatial parameters
crs = "EPSG:4326"  # WGS84 coordinate system
transform = (
    0.001,    # pixel size in x direction
    0.0,      # rotation (0 if image is "north up")
    23.5,     # x coordinate of upper left corner
    0.0,      # rotation (0 if image is "north up")
    -0.001,   # pixel size in y direction (negative since y decreases with increasing row)
    45.2      # y coordinate of upper left corner
)

repo.add_single_tile_from_array(
    array=array,
    satellite_name="Sentinel-2",
    date_origin=datetime.now(),  # You can use datetime objects
    crs=crs,
    transform=transform
)

In [4]:
# Create a sample array (4 bands, 256x256 pixels)
array = np.random.rand(4, 256, 256)

# Define the spatial parameters
crs = "EPSG:4326"  # WGS84 coordinate system

# Define the affine transform
# This example creates a tile at (23.5°E, 45.2°N) with 0.001° pixel size
transform = (
    0.001,    # a: pixel width in degrees
    0.0,      # b: row rotation (0 for north-up image)
    23.5,     # c: x-coordinate of upper-left corner
    0.0,      # d: column rotation (0 for north-up image)
    -0.001,   # e: pixel height in degrees (negative because y decreases with increasing row)
    45.2      # f: y-coordinate of upper-left corner
)

# Add the tile
tile = repo.add_single_tile_from_array(
    array=array,
    satellite_name="Sentinel-2",
    date_origin=datetime.now(),
    crs=crs,
    transform=transform
)

print(f"Created tile with ID: {tile['id'].values[0]}")

Created tile with ID: 2


In [3]:
# Example 1: Split a single raster into patches
# Define a validator function (optional)
def valid_patch(patch):
    """Return True if patch is valid (no NaN values and >50% non-zero pixels)"""
    return not np.any(np.isnan(patch)) and np.mean(patch != 0) > 0.5

# Save patches from a raster
tiles = repo.save_raster_patches(
    raster_path="sample_rasters/D2-005-2023SPneo.tif",
    satellite_name="Sentinel-2",
    date_origin=datetime.now(),
    patch_size=256,  # Size of patches in pixels
    validator=valid_patch,  # Optional: skip invalid patches
    stride=128,  # Optional: overlap patches by 128 pixels
    file_format='tif'
)

print(f"Created {len(tiles)} tiles")

Creating patches:   1%|          | 1/103 [00:03<06:31,  3.84s/it]


KeyboardInterrupt: 

In [4]:
repo.save_changes()

In [3]:
# Optional validator function
def valid_mask(mask_data):
    return np.any(mask_data != 0)  # At least some non-zero pixels

masks = repo.create_tile_masks_from_raster(
    mask_path="sample_rasters/D2-005-2023SPneo.tif",
    task="field_delineation",  # or other valid task type
    mask_type="binary",        # or other mask type
    file_format="tif"
)

Mask bounds: (176000.0, 4796000.0, 180000.2, 4800000.2)

Executing query: 
            SELECT * FROM tile 
            WHERE CAST(json_extract_string(ST_AsGeoJSON(bounds), '$.coordinates[0][0][0]') AS DOUBLE) <= 180000.2  -- mask xmax
            AND CAST(json_extract_string(ST_AsGeoJSON(bounds), '$.coordinates[0][2][0]') AS DOUBLE) >= 176000.0    -- mask xmin
            AND CAST(json_extract_string(ST_AsGeoJSON(bounds), '$.coordinates[0][0][1]') AS DOUBLE) <= 4800000.2    -- mask ymax
            AND CAST(json_extract_string(ST_AsGeoJSON(bounds), '$.coordinates[0][2][1]') AS DOUBLE) >= 4796000.0    -- mask ymin
            


Creating tile masks:   3%|▎         | 4/124 [00:00<00:03, 33.47it/s]


Processing tile 1:
Tile path: uploads\3a6a0c93-cf18-482f-bcf4-3920db6e2832.tif
Tile bounds from file: BoundingBox(left=176000.0, bottom=4799923.4, right=176076.8, top=4800000.2)
Tile transform (formatted): '0.3,0.0,176000.0,0.0,-0.3,4800000.2'

Processing tile 2:
Tile path: uploads\2fa2ad77-3396-416a-a3cf-6a669047ecf6.tif
Tile bounds from file: BoundingBox(left=176038.4, bottom=4799923.4, right=176115.19999999998, top=4800000.2)
Tile transform (formatted): '0.3,0.0,176038.4,0.0,-0.3,4800000.2'

Processing tile 3:
Tile path: uploads\32d6b3d8-02c9-4962-b7a7-22802567afde.tif
Tile bounds from file: BoundingBox(left=176076.8, bottom=4799923.4, right=176153.59999999998, top=4800000.2)
Tile transform (formatted): '0.3,0.0,176076.8,0.0,-0.3,4800000.2'

Processing tile 4:
Tile path: uploads\1a5cac18-e4f2-4463-9470-92f4bc928c85.tif
Tile bounds from file: BoundingBox(left=176115.2, bottom=4799923.4, right=176192.0, top=4800000.2)
Tile transform (formatted): '0.3,0.0,176115.2,0.0,-0.3,4800000.2'


Creating tile masks:  11%|█▏        | 14/124 [00:00<00:02, 42.60it/s]


Processing tile 8:
Tile path: uploads\9998fa65-385d-4f2c-9b38-0f855f703d84.tif
Tile bounds from file: BoundingBox(left=176268.8, bottom=4799923.4, right=176345.59999999998, top=4800000.2)
Tile transform (formatted): '0.3,0.0,176268.8,0.0,-0.3,4800000.2'

Processing tile 9:
Tile path: uploads\41e511fd-c91f-48e9-a227-ab9de2831d85.tif
Tile bounds from file: BoundingBox(left=176307.2, bottom=4799923.4, right=176384.0, top=4800000.2)
Tile transform (formatted): '0.3,0.0,176307.2,0.0,-0.3,4800000.2'

Processing tile 10:
Tile path: uploads\5484eefa-bbed-4c84-9d15-af2dec08851f.tif
Tile bounds from file: BoundingBox(left=176345.6, bottom=4799923.4, right=176422.4, top=4800000.2)
Tile transform (formatted): '0.3,0.0,176345.6,0.0,-0.3,4800000.2'

Processing tile 11:
Tile path: uploads\3c7465b9-3d0f-44f5-96eb-03627dd534ca.tif
Tile bounds from file: BoundingBox(left=176384.0, bottom=4799923.4, right=176460.8, top=4800000.2)
Tile transform (formatted): '0.3,0.0,176384.0,0.0,-0.3,4800000.2'

Process

...


Processing tile 109:
Tile path: uploads\c73bae7d-e2a1-4e51-8f1d-2071ce171c86.tif
Tile bounds from file: BoundingBox(left=176192.0, bottom=4799885.0, right=176268.8, top=4799961.8)
Tile transform (formatted): '0.3,0.0,176192.0,0.0,-0.3,4799961.8'

Processing tile 110:
Tile path: uploads\5340c16f-2752-453c-93d9-fc3ade6ca7a1.tif
Tile bounds from file: BoundingBox(left=176230.4, bottom=4799885.0, right=176307.19999999998, top=4799961.8)
Tile transform (formatted): '0.3,0.0,176230.4,0.0,-0.3,4799961.8'

Processing tile 111:
Tile path: uploads\acabfe90-f3ec-459c-aa4b-a45524291236.tif
Tile bounds from file: BoundingBox(left=176268.8, bottom=4799885.0, right=176345.59999999998, top=4799961.8)
Tile transform (formatted): '0.3,0.0,176268.8,0.0,-0.3,4799961.8'

Processing tile 112:
Tile path: uploads\5a87455a-ec5c-4256-8224-68bf85bee50d.tif
Tile bounds from file: BoundingBox(left=176307.2, bottom=4799885.0, right=176384.0, top=4799961.8)
Tile transform (formatted): '0.3,0.0,176307.2,0.0,-0.3,479

Creating tile masks:  98%|█████████▊| 122/124 [00:03<00:00, 35.04it/s]


Processing tile 117:
Tile path: uploads\860083e6-f5e7-46b4-97ff-82613ecc8581.tif
Tile bounds from file: BoundingBox(left=176499.2, bottom=4799885.0, right=176576.0, top=4799961.8)
Tile transform (formatted): '0.3,0.0,176499.2,0.0,-0.3,4799961.8'

Processing tile 118:
Tile path: uploads\a14748cd-eaba-4198-83c8-a3b7e2bc489c.tif
Tile bounds from file: BoundingBox(left=176537.6, bottom=4799885.0, right=176614.4, top=4799961.8)
Tile transform (formatted): '0.3,0.0,176537.6,0.0,-0.3,4799961.8'

Processing tile 119:
Tile path: uploads\7f5673f3-6252-4100-b2a6-6c4fcb3ba182.tif
Tile bounds from file: BoundingBox(left=176576.0, bottom=4799885.0, right=176652.8, top=4799961.8)
Tile transform (formatted): '0.3,0.0,176576.0,0.0,-0.3,4799961.8'

Processing tile 120:
Tile path: uploads\4d195fd7-984b-4e3f-841a-28bef9203529.tif
Tile bounds from file: BoundingBox(left=176614.4, bottom=4799885.0, right=176691.19999999998, top=4799961.8)
Tile transform (formatted): '0.3,0.0,176614.4,0.0,-0.3,4799961.8'

P

Creating tile masks: 100%|██████████| 124/124 [00:03<00:00, 37.12it/s]


Processing tile 124:
Tile path: uploads\d39e98fc-9ce4-4942-b361-b23875b79436.tif
Tile bounds from file: BoundingBox(left=176768.0, bottom=4799885.0, right=176844.8, top=4799961.8)
Tile transform (formatted): '0.3,0.0,176768.0,0.0,-0.3,4799961.8'


In [ ]:
repo = TileBankRepository(save_dir="uploads")